In [117]:
from __future__ import print_function
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [118]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.0.0-bin-hadoop3.2\\'

In [54]:
spark = SparkSession.builder.appName("Pyspark ML Algorithms").getOrCreate()

In [55]:
dataframe = spark.read.csv("Admission_Prediction.csv",header=True)

In [56]:
type (dataframe)

pyspark.sql.dataframe.DataFrame

In [57]:
dataframe.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [58]:
dataframe.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



In [59]:
from pyspark.sql.functions import col
new_dataframe = dataframe.select(*(col(c).cast("float").alias(c) for c in dataframe.columns))

In [60]:
new_dataframe.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [61]:
from pyspark.sql.functions import col, count, isnan, when

In [119]:
for c in new_dataframe.columns:
    print(c)

GRE Score
TOEFL Score
University Rating
SOP
LOR
CGPA
Research
Chance of Admit


In [120]:
#checking for null ir nan type values in our columns
new_dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in new_dataframe.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [64]:
from pyspark.ml.feature import Imputer

In [121]:
imputer = Imputer(inputCols=["GRE Score", "TOEFL Score","University Rating"], 
                  outputCols=["GRE Score", "TOEFL Score","University Rating"])
model = imputer.fit(new_dataframe)

imputed_data = model.transform(new_data)

In [122]:
imputed_data

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR: float, CGPA: float, Research: float, Chance of Admit: float]

In [123]:
#checking for null ir nan type values in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [124]:
features = imputed_data.drop('Chance of Admit')

In [125]:
features

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR: float, CGPA: float, Research: float]

In [126]:
#let's assemble our features together using vectorAssembler
assembler = VectorAssembler( inputCols=features.columns,outputCol="features")

In [127]:
output = assembler.transform(imputed_data)

## Linear Regressor

In [128]:
output= output.select("features", "Chance of Admit")

In [129]:
output = assembler.transform(imputed_data)

In [130]:
output= output.select("features", "Chance of Admit")

In [131]:
train_df,test_df = output.randomSplit([0.7, 0.3])

In [132]:
train_df.show()
test_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[290.0,104.0,4.0,...|           0.45|
|[294.0,93.0,1.0,1...|           0.46|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,99.0,2.0,2...|           0.61|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,98.0,2.0,2...|           0.59|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,98.0,2.0,4...|           0.34|
|[298.0,99.0,1.0,1...|           0.53|
|[298.0,100.0,3.0,...|           0.58|
+--------------------+---------------+
only showing top 20 rows

+--------------------+---------------+

In [46]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit')
linear_model = lin_reg.fit(train_df)


In [47]:
print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))

Coefficients: [0.0018186171380833336,0.0028664355141501128,0.008170168538492182,-0.0016601576552002632,0.018654193850216265,0.12048031637215754,0.022763298019048573]
Intercept: -1.29343488377689


In [48]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("r2: %f" % trainSummary.r2)

RMSE: 0.060150
r2: 0.823635


In [49]:
# prediction

predictions = linear_model.transform(test_df)
predictions.select("prediction","Chance of Admit","features").show()

+-------------------+---------------+--------------------+
|         prediction|Chance of Admit|            features|
+-------------------+---------------+--------------------+
|0.43754052197663196|           0.46|[294.0,93.0,1.0,1...|
|0.41925217060191566|           0.46|[295.0,93.0,1.0,2...|
| 0.5196569410324312|            0.6|[296.0,101.0,1.0,...|
| 0.5198416468765705|           0.59|[297.0,98.0,2.0,2...|
| 0.5162749597886092|           0.44|[298.0,98.0,2.0,1...|
|    0.6107152518632|           0.58|[298.0,100.0,3.0,...|
| 0.5361568013667373|           0.54|[298.0,101.0,2.0,...|
| 0.5997541004579252|           0.53|[298.0,101.0,4.0,...|
|  0.671707378803112|           0.69|[298.0,105.0,3.0,...|
| 0.6644079297638907|           0.56|[299.0,102.0,3.0,...|
| 0.5941863124916837|           0.63|[300.0,102.0,3.0,...|
|  0.536690875900222|           0.58|[300.0,105.0,1.0,...|
| 0.5859121962674032|           0.67|[301.0,98.0,1.0,2...|
|  0.636236438248946|           0.68|[301.0,99.0,3.0,2..

In [100]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Chance of Admit",metricName="r2")
print("R Squared (R2) on test data =", pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.7700269878844765


## Random Forest Regressor

In [134]:
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(output)

IllegalArgumentException: requirement failed: Column features already exists.

In [102]:
featureIndexer = featureIndexer.transform(output)

In [135]:
new_indexed_data = featureIndexer.select("indexedFeatures", "Chance of Admit")

In [104]:
training, test = new_indexed_data.randomSplit([0.7, 0.3])

In [105]:
training.show()

+--------------------+---------------+
|     indexedFeatures|Chance of Admit|
+--------------------+---------------+
|[290.0,104.0,4.0,...|           0.45|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,99.0,2.0,2...|           0.57|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,99.0,2.0,3...|           0.47|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,4...|           0.34|
|[298.0,99.0,1.0,1...|           0.53|
|[298.0,100.0,3.0,...|           0.58|
|[298.0,101.0,2.0,...|           0.54|
+--------------------+---------------+
only showing top 20 rows



In [106]:
test.show()

+--------------------+---------------+
|     indexedFeatures|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[294.0,93.0,1.0,1...|           0.46|
|[295.0,96.0,2.0,1...|           0.47|
|[296.0,99.0,2.0,2...|           0.61|
|[297.0,96.0,2.0,2...|           0.34|
|[297.0,98.0,2.0,2...|           0.59|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,101.0,4.0,...|           0.53|
|[298.0,105.0,3.0,...|           0.69|
|[298.0,107.187751...|           0.46|
|[299.0,96.0,2.0,1...|           0.54|
|[299.0,100.0,3.0,...|           0.42|
|[300.0,100.0,3.0,...|           0.64|
|[300.0,105.0,1.0,...|           0.58|
|[301.0,96.0,1.0,3...|           0.54|
|[301.0,97.0,2.0,3...|           0.44|
|[301.0,100.0,3.0,...|           0.67|
|[301.0,104.0,2.0,...|           0.68|
|[301.0,104.0,3.0,...|           0.68|
+--------------------+---------------+
only showing top 20 rows



In [107]:
random_forest_reg = RandomForestRegressor(featuresCol="indexedFeatures",labelCol="Chance of Admit" )

In [133]:
# Train model.  This also runs the indexer.
model = random_forest_reg.fit(train_df)

IllegalArgumentException: indexedFeatures does not exist. Available: features, Chance of Admit

In [109]:
# Make predictions.
predictions = model.transform(test)

In [110]:
predictions.show()

+--------------------+---------------+-------------------+
|     indexedFeatures|Chance of Admit|         prediction|
+--------------------+---------------+-------------------+
|[290.0,100.0,1.0,...|           0.47| 0.5094936201007437|
|[294.0,93.0,1.0,1...|           0.46|0.48588058168662174|
|[295.0,96.0,2.0,1...|           0.47|0.49061391507558605|
|[296.0,99.0,2.0,2...|           0.61| 0.5740165672682737|
|[297.0,96.0,2.0,2...|           0.34|0.48234620826912505|
|[297.0,98.0,2.0,2...|           0.59| 0.5224304593588183|
|[298.0,92.0,1.0,2...|           0.51| 0.5070473164399992|
|[298.0,98.0,2.0,1...|           0.44| 0.5118082880265786|
|[298.0,101.0,4.0,...|           0.53| 0.5917534679340165|
|[298.0,105.0,3.0,...|           0.69| 0.6536050774059645|
|[298.0,107.187751...|           0.46| 0.5030141599807718|
|[299.0,96.0,2.0,1...|           0.54| 0.5097316906929543|
|[299.0,100.0,3.0,...|           0.42| 0.5497166284245074|
|[300.0,100.0,3.0,...|           0.64| 0.675628206762394

In [114]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit", predictionCol="prediction", metricName="rmse")
print ("Root Mean Squared Error (RMSE) on test data = ",evaluator.evaluate(predictions))

Root Mean Squared Error (RMSE) on test data =  0.05799074549599089


In [115]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit", predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.8353874931296166
